<p align="center">
Universidade Federal de Alagoas</br>

Instituto de Computação</br>

</p>

#Trabalho Prático AB2

Disciplina: Sistemas Distribuídos

Semestre letivo: 2016.1

Professor: André Lage Freitas

## Especificação

**Importante**. Leia atentamente essa seção pois só serão aceitos trabalhos que obedecerem as a especificações a seguir. 

Baseado(a) no exemplo a seguir de implementação do B-multicast em Julia, façam os exercícios descritos nesse documento.

In [28]:
function B_multicast(g,m,B_deliver)
    for p in g
        remotecall(p, B_deliver, m)
    end
end
 
# Adding 3 processes (workers)
#addprocs(3)

# My group is composed by all workers
g = workers()

times = Any[]
# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
for i in 1:51
    tic()
    B_multicast(g,"oi!",println)
    push!(times,toc())
end

println(times)

	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
elapsed time: 	From worker 9:	oi!
0.03302788 seconds
	From worker 10:	oi!
	From worker 11:	oi!
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
0.0	From worker 11:	oi!
00621079 seconds
	From worker 2:	oi!
	From worker 4:	oi!
	From worker 3:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
0.0	From worker 11:	oi!
0060882 seconds
	From worker 2:	oi!
	From worker 4:	oi!
	From worker 3:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
0.0	From worker 11:	oi!
00603692 seconds
	From worker 2:	oi!
	From worker 4:	oi!
	From worker 3:	oi

### Exercício 1: Sincronismo (2,0 pontos)

Adicione 10 processos com o comando `addprocs` abaixo. 

_Atenção_: sua instalação local de Julia pode não conseguir adicionar 10 processos, use a [JuliaBox](https://juliabox.org).

In [2]:
rmprocs(procs())
addprocs(10)

10-element Array{Any,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

Modifique a implementação do B-multicast para utilizar communicação síncrona.

In [27]:
function sync_B_multicast(g,m,B_deliver)
    for p in g
        remotecall_wait(p, B_deliver, m)
    end
end

# My group is composed by all workers
g = workers()

# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
times = Any[]
# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
for i in 1:51
    tic()
    B_multicast(g,"oi!",println)
    push!(times,toc())
end
println(times)

	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
elapsed time: 	From worker 9:	oi!
0.	From worker 10:	oi!
001516239 seconds
	From worker 11:	oi!
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
0.0	From worker 11:	oi!
00866617 seconds
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
	From worker 11:	oi!
0.00094734 seconds
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
elapsed time: 	From worker 10:	oi!
0.	From worker 11:	oi!
000848192 seconds
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	o

Execute as duas versões do B-multicast (síncrona e assíncrona) e compare as saídas.

A ordem das mensagens das saídas foram as mesmas para ambas versões do B-multicast? Por quê?

**Resposta**: _Não, pois ao tornar o B_multicast síncrono, a cada chamada tivemos que aguardar sua resposta, então a saída ficou ordenada de acordo com as chamadas dos processos. No caso do B_multicast assíncrono observamos que não há uma ordem na saída dos processos, aquele que for encerrando suas atividades primeiro vai imprimindo a saída. _



### Exercício 2: Análise de desempenho do B-multicast (2,0 pontos)

Meça o tempo de execução duas versões do B-multicast (síncrona e assíncrona). Dica: utilize a macro `@time`.

* Execute 50 vezes essa medição e construa um gráfico em forma de barras:
	* cada barra representando cada versão do B-multicast;
	* calcule e coloque no gráfico o [intervalo de confiança](https://en.wikipedia.org/wiki/Confidence_interval) com nível de 95%.
    
**Gráfico**:   
    
    
* Qual das duas versões teve o menor tempo de execução? Por quê?

**Resposta**: _em itálico_



### Exercício 3: Análise de desempenho do R-multicast (3,0 pontos)

Implemente duas versões do algoritmo R-multicast: uma assíncrona e outra síncrona.

* R-multicast assíncrono

Execute as duas versões do B-multicast (síncrona e assíncrona) e compare as saídas.

A ordem das mensagens das saídas foram as mesmas para ambas versões do B-multicast? Por quê?

**Resposta**: _Não, pois ao tornar o B_multicast síncrono, a cada chamada tivemos que aguardar sua resposta, então a saída ficou ordenada de acordo com as chamadas dos processos. No caso do B_multicast assíncrono observamos que não há uma ordem na saída dos processos, aquele que for encerrando suas atividades primeiro vai imprimindo a saída. _



### Exercício 2: Análise de desempenho do B-multicast (2,0 pontos)

Meça o tempo de execução duas versões do B-multicast (síncrona e assíncrona). Dica: utilize a macro `@time`.

* Execute 50 vezes essa medição e construa um gráfico em forma de barras:
	* cada barra representando cada versão do B-multicast;
	* calcule e coloque no gráfico o [intervalo de confiança](https://en.wikipedia.org/wiki/Confidence_interval) com nível de 95%.
    
**Gráfico**:    
    
    
* Qual das duas versões teve o menor tempo de execução? Por quê?

**Resposta**: _em itálico_



### Exercício 3: Análise de desempenho do R-multicast (3,0 pontos)

Implemente duas versões do algoritmo R-multicast: uma assíncrona e outra síncrona.

* R-multicast assíncrono

In [6]:
# Array keeping a message history. 
@everywhere received = Any[]

# My group is composed by all workers
g = workers()

# Function for delivering message and assuring everyone in the group will receive.
function sync_B_multicast(p,g,m,R_deliver)
    if !(p in received)
        push!(received, p)
        remotecall_wait(p, R_deliver, m)
        sync_R_multicast(g, m, R_deliver)
    end
end

function sync_R_multicast(g,m, R_deliver)
    #message = Dict("message" => m, "pid" => myid(), "flag" => true)
    push!(received,m)
    for p in g
        sync_B_multicast(p,g,m,R_deliver) 
    end
end

# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
sync_R_multicast(g,"oi!",println)

	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
	From worker 10:	oi!
	From worker 11:	oi!


* R-multicast assíncrono

In [13]:
# Array keeping a message history. 
@everywhere received = Any[]

# My group is composed by all workers
g = workers()

# Function for delivering message and assuring everyone in the group will receive.
function B_multicast(p,g,m,R_deliver)
    if !(p in received)
        push!(received, p)
        remotecall(p, R_deliver, m)
        R_multicast(g, m, R_deliver)
    end
end

function R_multicast(g,m, R_deliver)
    #message = Dict("message" => m, "pid" => myid(), "flag" => true)
    #push!(received,m)
    for p in g
        B_multicast(p,g,m,R_deliver) 
    end
end

# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
R_multicast(g,"oi!",println)

	From worker 10:	oi!
	From worker 11:	oi!
	From worker 3:	oi!
	From worker 6:	oi!
	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!


Faça a mesma análise de desempenho para o R-multicast que você fez no Exercício 2.


**Gráfico**: _por aqui_    
    
    
* Qual das duas versões (R-multicast assíncrono e assíncrono) teve o menor tempo de execução? Por quê?

**Resposta**: _em itálico_



### Exercício 4: Análise de desempenho: B-multicast vs. R-multicast (3,0 pontos)

Para cada uma das comparações abaixo, qual dos algoritmos teve o menor tempo de execução? Por quê?

* B-multicast assíncrono vs. R-multicast assíncrono.
    

**Resposta**: _em itálico_    
    
* B-multicast síncrono vs. R-multicast síncrono.


**Resposta**: _em itálico_

## Entrega

As respostas deverão ser entregues nos devidos campos desse arquivo, ou seja, em [Markdown](https://en.wikipedia.org/wiki/Markdown) e no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl).

O programa deve ser implementado na linguagem de programação **Julia**. Seu trabalho será testado na [JuliaBox](https://juliabox.org), portanto teste seu arquivo IJulia na JulizBox antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente por e-mail** para o seguinte endereço: **prof.alage** (no) **gmail com**


Será registrada como data de recebimento do trabalho a data indicada pelo servidor de e-mail do Professor. 

A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos ou se o e-mail não chegar à caixa de entrada de e-mail do Professor.

### Prazos

O prazo de entrega sem atraso é o dia **12/10/16**.

Trabalhos entregues com atraso serão aceitos, no máximo, até o o dia **16/10/16**. Serão descontados 5% da nota obtida por dia de atraso.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.